In [9]:
import numpy as np
import matplotlib.pyplot as plt

## Inisialisasi Data

In [29]:
m1 = float(input("Massa Pendulum 1: "))
m2 = float(input("Massa Pendulum 2: "))
L1 = float(input("Panjang Tali 1: "))
L2 = float(input("Panjang Tali 2: "))
teta1 = float(input("Sudut Pendulum 1: "))
teta2 = float(input("Sudut Pendulum 2: "))
dt = float(input("Step Size: "))
teta= np.array([[teta1],[teta2]])
tetadot = np.array([[0], [0]])


## Kalkulasi Inersia

In [30]:
i1 = m1*(L1**2)
i2 = m2*(L2**2)


## Posisi Garis dan Pendulum

In [13]:
def pos():
    global L1, L2, teta
    pos_garis = np.array([[[0, 0],
                           [L1*np.sin(teta[0,0]), (-L1*np.cos(teta[0,0]))]],
                          [[L1*np.sin(teta[0,0]), -L1*np.cos(teta[0,0])],
                           [L1*np.sin(teta[0,0])+L2*np.sin(teta[1,0]), -(L1*np.cos(teta[0,0])+L2*np.cos(teta[1,0]))]]])
    pos_massa = np.array([[1/2*L1*np.sin(teta[0,0]), -1/2*L1*np.cos(teta[0,0])],
                          [L1*np.sin(teta[0,0])+1/2*L2*np.sin(teta[1,0]), -(L1*np.cos(teta[0,0])+1/2*L2*np.cos(teta[1,0]))]])
    return pos_garis, pos_massa


## Fungsi Pendulum

In [49]:
def inverse(m):
    determinant = m[0][0]*m[1][1]-m[0][1]*m[1][0]
    # special case for 2x2 matrix:
    return np.array([[m[1][1], -1*m[0][1]],
                     [-1*m[1][0], m[0][0]]])/determinant


def pendulum(teta,tetadot):
    global m1, m2, L1, L2, i1, i2
    g = 9.8
    tau = np.array([[0], [0]])
    M = np.array([[(1/4*m1+m2)*(L1**2)+i1, 1/2*m2*L1*L2*np.cos(teta[0,0]-teta[1,0])],
                 [1/2*m2*L1*L2*np.cos(teta[0,0]-teta[1,0]), 1/4*m2*(L2**2)+i2]])
    Minv = inverse(M)
    matrix1 = np.dot(Minv, tau)
    # print(matrix1.shape)

    C = np.array([[1/2*m2*L1*L2*np.sin(teta[0,0]-teta[1,0])*tetadot[1,0], -1/2*m2*L1*L2*np.sin(teta[0,0]-teta[1,0])*(tetadot[0,0]-tetadot[1,0])],
                 [-1/2*m2*L1*L2*np.sin(teta[0,0]-teta[1,0])*(tetadot[0,0]-tetadot[1,0]), -1/2*m2*L1*L2*np.sin(teta[0,0]-teta[1,0])*tetadot[0,0]]])
    A = np.dot(Minv, C)
    matrix2 = np.dot(A, tetadot)
    # print(matrix2.shape)

    G = np.array([[(1/2*m1+m2)*g*L1*np.sin(teta[0,0])],
                  [1/2*m2*g*L2*np.sin(teta[1,0])]])
    matrix3 = np.dot(Minv, G)
    # print(matrix3.shape)

    return matrix1-matrix2-matrix3


## Fungsi Runge Kutta

In [51]:
def rungekuttaupdate(step):
    global teta,tetadot
    tetadotdot = pendulum(teta,tetadot)
    k1 = step/2*tetadotdot
    k2 = step/2*pendulum(teta+(step/2)*(tetadot+k1/2),tetadot+k1)
    k3 = step/2*pendulum(teta+(step/2)*(tetadot+k1/2),tetadot+k2)
    k4 = step/2*pendulum(teta+step*(tetadot+k3),tetadot+2*k3)
    teta+=step*(tetadot+(k1+k2+k3)/3)
    tetadot+=(k1+2*k2+2*k3+k4)/3

## Animasi

In [52]:
def plotanimation(step):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-52-d01d2106b872>, line 2)